# Pretty Printing Helper

In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Creating Client

In [3]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
_:bool = load_dotenv(find_dotenv())
client:OpenAI = OpenAI()

# Upload the file and Create an Assistant

In [4]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose='assistants'
)

print(file)

FileObject(id='file-ykOsN482Fenk3IUG5MTJQj46', bytes=48802, created_at=1701430009, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


# Create an Assistant

In [5]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

# Create a thread

In [6]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

# Add a Message to a Thread

In [7]:
from openai.types.beta.threads.thread_message import ThreadMessage

threadMessage: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is Zia U. Khan?"
)

# Run the Assistant

In [8]:
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Check the Run status

In [9]:
# run: Run = client.beta.threads.runs.retrieve(
#     thread_id=thread.id,
#     run_id=run.id
# )
# show_json(run)

import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_UHEz9nyPwMoRAU5yfJqjdBAz',
 'assistant_id': 'asst_d01OXRE00QGCUvi7cgajwfSV',
 'cancelled_at': None,
 'completed_at': 1701430028,
 'created_at': 1701430023,
 'expires_at': None,
 'failed_at': None,
 'file_ids': ['file-ykOsN482Fenk3IUG5MTJQj46'],
 'instructions': 'You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1701430023,
 'status': 'completed',
 'thread_id': 'thread_OwRljOkZcCH8enU8HVaolIMs',
 'tools': [{'type': 'retrieval'}]}

# Display the Assistant's Response

In [10]:
messages: list[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user: What is Zia U. Khan?
assistant: Zia U. Khan is a highly accomplished individual with a diverse set of qualifications, including an MS, MBA, MAC, MA, CPA, and CMA. He is the CEO and Founder of Panacloud (Pvt.) Ltd. and Panaverse LLC, as well as the COO and Founder of PIAIC (Presidential Initiative for Artificial Intelligence and Computing) and the Governor Sindh Initiative for Artificial Intelligence, Web 3.0 & Metaverse【7†source】.
